In [12]:
from __future__ import annotations

import json
import random
from pathlib import Path

In [16]:
def validate_parameters(output_weight, quality_params):
    """Validate the quality parameters and output weight."""
    if output_weight <= 0:
        raise ValueError("Output weight must be greater than 0.")

    for param in quality_params:
        minimum = param.get("minimum")
        maximum = param.get("maximum")
        goal = param.get("goal")
        importance = param.get("importance")
        
        if minimum is None or maximum is None or goal is None or importance is None:
            raise ValueError("All quality parameters must include 'minimum', 'maximum', 'goal', and 'importance'.")
        
        if not (minimum < goal <= maximum):
            raise ValueError(f"Invalid quality parameter values: minimum ({minimum}), goal ({goal}), maximum ({maximum}).")
        
        if importance <= 0:
            raise ValueError("Importance must be greater than 0.")


def generate_quality_values(quality_params):
    """Generate random quality values for stockpiles based on output quality parameters."""
    stock_quality = []
    for param in quality_params:
        quality = {
            "parameter": param["parameter"],
            "value": round(random.uniform(param["minimum"], param["maximum"]), 2)
        }
        stock_quality.append(quality)
    return stock_quality


def generate_problem_data(num_stockpiles, num_engines, can_access_all, output_weight, quality_params):
    """Generate the problem data in JSON format based on the given parameters."""
    
    # Validate the output weight and quality parameters
    validate_parameters(output_weight, quality_params)
    
    # Create stockpiles with valid quality parameters
    stockpiles = []
    total_weight = 0
    
    for i in range(1, num_stockpiles + 1):
        weight_ini = int(
            round(
                random.uniform(
                    output_weight / num_stockpiles,
                    output_weight / (num_stockpiles - i + 1)
                ),
                0,
            )
        )
        total_weight += weight_ini
        stockpile = {
            "id": i,
            "position": i - 1,
            "yard": random.randint(1, num_stockpiles // 5 + 1),  # Randomly assign to a yard
            "rails": [1, 2] if can_access_all else [random.randint(1, 2)],
            "capacity": weight_ini,  # Assuming capacity is double the initial weight
            "weightIni": weight_ini,
            "qualityIni": generate_quality_values(quality_params)
        }
        stockpiles.append(stockpile)
    
    # Ensure the total weight exceeds the required output weight
    if total_weight <= output_weight:
        raise ValueError("Sum of all stockpile weights must be greater than the specified output weight.")
    
    # Generate engines
    engines = []
    for j in range(1, num_engines + 1):
        engine = {
            "id": j,
            "speedStack": 0.0,
            "speedReclaim": random.randint(2500, 4000),
            "posIni": random.randint(1, num_stockpiles),
            "rail": random.randint(1, 2),
            "yards": list(range(1, num_stockpiles // 5 + 2)) if can_access_all else [random.randint(1, num_stockpiles // 5 + 1)]
        }
        engines.append(engine)
    
    # Generate distances and travel times between stockpiles
    distances_travel = []
    time_travel = []
    for i in range(num_stockpiles):
        distances_row = []
        time_row = []
        for j in range(num_stockpiles):
            if i == j:
                distances_row.append(0.0)
                time_row.append(0.06)
            else:
                distance = round(random.uniform(0.05, 1.5), 2)
                distances_row.append(distance)
                time_row.append(round(distance / random.uniform(1, 2), 2))  # 
                # Random time based on distance
        distances_travel.append(distances_row)
        time_travel.append(time_row)
    
    # Prepare the output JSON structure
    problem_data = {
        "info": ["Generated_Instance", 1000, 1],
        "stockpiles": stockpiles,
        "engines": engines,
        "inputs": [
            {
                "id": 1,
                "weight": 0.0,
                "quality": [{"parameter": param["parameter"], "value": param["goal"]} for param in quality_params],
                "time": 0
            }
        ],
        "outputs": [
            {
                "id": 1,
                "destination": 1,
                "weight": output_weight,
                "quality": quality_params,
                "time": 0
            }
        ],
        "distancesTravel": distances_travel,
        "timeTravel": time_travel
    }
    
    return problem_data


def save_new_instance(
    data: dict,
    folder_path: str | Path,
    name_pattern: str = "instance_*.json",
):
    _folder_path = Path(folder_path)
    if _folder_path.is_file():
        raise ValueError(
            "The folder path must be a path to an existing or to be created "
            f"directory, not a file: '{folder_path}'"
        )
    _folder_path.mkdir(exist_ok=True, parents=True)
    files_enumerations = []
    for filepath in _folder_path.glob(name_pattern):
        filename = filepath.with_suffix("").name
        number = "".join(
            character for character in filename if character.isnumeric()
        )
        if number != "":
            files_enumerations.append(number)

    next_enumeration = 1

    if len(files_enumerations) > 0:
        files_enumerations = [int(number) for number in files_enumerations]
        last_enumeration = max(files_enumerations)
        next_enumeration = last_enumeration + 1

    new_filename = name_pattern.replace("*", str(next_enumeration))
    new_filepath = _folder_path.joinpath(new_filename).with_suffix(".json")

    while new_filepath.is_file():
        next_enumeration += 1
        new_filename = name_pattern.replace("*", str(next_enumeration))
        new_filepath = _folder_path.joinpath(new_filename).with_suffix(".json")

    with open(str(new_filepath), "w", encoding="utf-8") as fp:
        json.dump(data, fp, indent=2, allow_nan=False)

    print(f"Successfully saved generated data to: '{new_filepath}'")

In [14]:
# Example usage
quality_params_example = [
    {"parameter": "Fe", "minimum": 60, "maximum": 100, "goal": 65, "importance": 10},
    {"parameter": "SiO2", "minimum": 2.8, "maximum": 5.8, "goal": 5.8, "importance": 1000},
    {"parameter": "Al2O3", "minimum": 2.5, "maximum": 4.9, "goal": 4.9, "importance": 100},
    {"parameter": "P", "minimum": 0.05, "maximum": 0.07, "goal": 0.07, "importance": 100},
    {"parameter": "+31.5", "minimum": 8, "maximum": 10, "goal": 10, "importance": 100},
    {"parameter": "-6.3", "minimum": 20, "maximum": 25, "goal": 25, "importance": 100}
]

generated_data = generate_problem_data(
    num_stockpiles=20,
    num_engines=3,
    can_access_all=True,
    output_weight=500_000,
    quality_params=quality_params_example
)

save_new_instance(generated_data, "../tests")

{'info': ['Generated_Instance', 1000, 1], 'stockpiles': [{'id': 1, 'position': 0, 'yard': 4, 'rails': [1, 2], 'capacity': 25000, 'weightIni': 25000, 'qualityIni': [{'parameter': 'Fe', 'value': 62.72}, {'parameter': 'SiO2', 'value': 3.96}, {'parameter': 'Al2O3', 'value': 4.77}, {'parameter': 'P', 'value': 0.06}, {'parameter': '+31.5', 'value': 9.89}, {'parameter': '-6.3', 'value': 22.3}]}, {'id': 2, 'position': 1, 'yard': 3, 'rails': [1, 2], 'capacity': 25503, 'weightIni': 25503, 'qualityIni': [{'parameter': 'Fe', 'value': 65.15}, {'parameter': 'SiO2', 'value': 4.52}, {'parameter': 'Al2O3', 'value': 3.92}, {'parameter': 'P', 'value': 0.07}, {'parameter': '+31.5', 'value': 8.22}, {'parameter': '-6.3', 'value': 20.67}]}, {'id': 3, 'position': 2, 'yard': 1, 'rails': [1, 2], 'capacity': 27167, 'weightIni': 27167, 'qualityIni': [{'parameter': 'Fe', 'value': 99.91}, {'parameter': 'SiO2', 'value': 5.5}, {'parameter': 'Al2O3', 'value': 4.15}, {'parameter': 'P', 'value': 0.07}, {'parameter': '+3

In [17]:
save_new_instance(generated_data, "../tests")

Successfully saved generated data to ../tests/instance_9.json!


In [5]:
with open("../tests/instance_8.json", "w", encoding="utf-8") as fp:
    json.dump(generated_data, fp, indent=4, allow_nan=False)